[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/antoniotrapote/chord-prediction-tfm/blob/main/anexos/notebooks/02_preprocesado/01_chord_norm_v1_mus21.ipynb)
[![View on GitHub](https://img.shields.io/badge/View_on-GitHub-black?logo=github)](https://github.com/antoniotrapote/chord-prediction-tfm/blob/main/anexos/notebooks/02_preprocesado/01_chord_norm_v1_mus21.ipynb)

# Transcripción de acordes a notacion funcional utilizando la librería *music21*

In [1]:
import pandas as pd

USER = "antoniotrapote"
REPO = "chord-prediction-tfm"
BRANCH = "main"
PATH_IN_REPO = "anexos/data/songdb.csv"
URL = f"https://raw.githubusercontent.com/{USER}/{REPO}/{BRANCH}/{PATH_IN_REPO}"

# Cargar el archivo
df = pd.read_csv(URL)

df.head()

,title,composedby,key,timesig,bars,chordprog
0,Lullaby of Birdland,George Shearing,Ab,4 4,32,Fm7 Dm7b5 | G7b9 C7b9 | Fm7 DbM7 | Bbm7 Eb7 |\...
1,It's A Most Unusual Day,Jimmy McHugh and HYarold Adamson,G,3 4,72,F#/G F#/G G | Em7 | Am7 | D7 |\nF#/G F#/G G | ...
2,Jump Monk,Charles Mingus,Ab,4 4,54,Fm7 DbM7 | G7b5 C7 | Fm7 DbM7 | G7b5 C7 |\nFm7...
3,Nuages,Django Reinhardt and Jacques Larme,G,4 4,32,Bbm7 Eb7 | Am7b5 D7b9 | G6 Am7 | Bm7 |\nBbm7 E...
4,Love Me Do,John Lennon and Paul McCartney,G,4 4,48,G | C | G | C |\nG | C | C | C |\nC | G | C | ...


In [2]:
from music21 import harmony, key as m21key, roman

def acorde_a_funcion(acorde_str, tonalidad_str):
    """
    Convierte un acorde a su notación funcional dentro de una tonalidad dada.
    Si hay algún error, se devuelve "?".
    """
    try:
        tonalidad = m21key.Key(tonalidad_str)
        acorde = harmony.ChordSymbol(acorde_str)
        rn = roman.romanNumeralFromChord(acorde, tonalidad)
        return rn.figure
    except Exception as e:
        return f"?"  # Si no se puede analizar el acorde


In [ ]:
# Prueba rápida
print(acorde_a_funcion("Gm7", "Cm"))      # Esperado: V7
print(acorde_a_funcion("Fm7", "Ab"))    # Esperado: i7
print(acorde_a_funcion("Dmaj7", "A"))   # Esperado: IVmaj7

v7
vi7
IV7


In [4]:
def transponer_progresion(prog_str, tonalidad_str):
    """
    Convierte una progresión completa a notación funcional,
    omitiendo los bajos alternativos (C/G → C).
    """
    compases = prog_str.strip().split("|")
    resultado = []

    for compas in compases:
        acordes = compas.strip().split()
        funciones = []
        for ac in acordes:
            ac_limpio = ac.split("/")[0]  # quitar bajo alternativo
            funciones.append(acorde_a_funcion(ac_limpio, tonalidad_str))
        resultado.append(" ".join(funciones))

    return " | ".join(resultado)

In [5]:
# Transposición de tres canciones de prueba
muestra = df[df["chordprog"].notna()].sample(3, random_state=33)

for _, fila in muestra.iterrows():
    titulo = fila["title"]
    tonalidad = fila["key"]
    prog_original = fila["chordprog"]
    prog_funcional = transponer_progresion(prog_original, tonalidad)

    print(f"🎵 {titulo} (Tonalidad: {tonalidad})")
    print(f"🔸 Original: {prog_original}")
    print(f"🔹 Funcional: {prog_funcional}")
    print("-" * 80)


🎵 I Hear A Rhapsody (Tonalidad: Eb)
🔸 Original: Cm9 F#7 | Fm7 Bb7 | EbM7 Ab9 | Gm7b5 C7b9 |
Fm7b5 | Bb7 | EbM7 | Dm7b5 G7alt |
Cm9 F#7 | Fm7 Bb7 | EbM7 Ab9 | Gm7b5 C7b9 |
Fm7b5 | Bb7 | EbM7 | Am7b5 D7 |
Gm7 | Am7b5 D7 | Gm | Cm7 F7 |
BbM7 | Fm7 | Ab7 | G7alt |
Cm9 F#7 | Fm7 Bb7 | EbM7 Gm7b5 | Db7 C7 |
Fm7b5 | Bb7 | EbM7 | Dm7b5 G7 |
🔹 Funcional: vi7532 #II75#3 | ii7 #V7 | ? #IV53b2 | iiiø7b53 VI75#3b2 | iiø7b53 | #V7 | ? | viiø7 ? | vi7532 #II75#3 | ii7 #V7 | ? #IV53b2 | iiiø7b53 VI75#3b2 | iiø7b53 | #V7 | ? | #ivø7 VII7#5#3 | iii7 | #ivø7 VII7#5#3 | iii | vi7 II75#3 | ? | ii7 | #IVb753 | ? | vi7532 #II75#3 | ii7 #V7 | ? iiiø7b53 | VII7#5#3 VI75#3 | iiø7b53 | #V7 | ? | viiø7 III75#3 | 
--------------------------------------------------------------------------------
🎵 Back Bay Shuffle (Tonalidad: F)
🔸 Original: F | F | C7 | C7 |
F F7 | Bb Bbm | F C13 | F C7 |
F | F | C7 | C7 |
F F7 | Bb Bbm | F C13 | F |
Am | Am | Am | E7 |
C | C | C | C C7 |
F | F | C7 | C7 |
F F7 | Bb Bbm | F C13 | F 

Observamos que hay problemas porque todas las canciones tienen asignada una tonalidad mayor en el dataset base

Así que vamos a probar a **transcribir una canción que está en tonalidad menor**, para ver si el transcriptor funciona correctamente.

In [7]:
titulo = 'Johnny Come Lately'
tonalidad = 'Gm'
progresion = df[df["title"] == titulo].iloc[0]["chordprog"]
progresion_funcional = transponer_progresion(progresion, tonalidad)

print(f"🎵 {titulo} (Tonalidad: {tonalidad})"
      f"\nProgresión: {progresion}"
      f"\nProgresión Funcional: {progresion_funcional}"
)

🎵 Johnny Come Lately (Tonalidad: Gm)
Progresión: Gm7 | Eb7 D7alt | Gm7 | Eb7 D7alt |
Gm7 | Eb7 D7alt | Gm7 | Eb7 D7alt |
Gm7 | Eb7 D7alt | Gm7 | Eb7 D7alt |
Gm7 | Eb7 D7alt | Gm7 | Fm7 Bb7 |
EbM7 Fm7 | Gm7 Fm7 | EbM7 Fm7 | Gm7 Fm7 |
EbM7 Fm7 | Gm7 C7 | Fm7 Bb7 | A7b9 D7alt |
Gm7 | Eb7 D7alt | Gm7 | Eb7 D7alt |
Gm7 | Eb7 D7alt | Gm7 | Eb7 D7alt |
Progresión Funcional: i7 | VI7#5#3 ? | i7 | VI7#5#3 ? | i7 | VI7#5#3 ? | i7 | VI7#5#3 ? | i7 | VI7#5#3 ? | i7 | VI7#5#3 ? | i7 | VI7#5#3 ? | i7 | bvii75b3 #IIIb753 | ? bvii75b3 | i7 bvii75b3 | ? bvii75b3 | i7 bvii75b3 | ? bvii75b3 | i7 IV75#3 | bvii75b3 #IIIb753 | II7#5#32 ? | i7 | VI7#5#3 ? | i7 | VI7#5#3 ? | i7 | VI7#5#3 ? | i7 | VI7#5#3 ? | 


In [8]:
# Aplicamos la función 'transponer_progresion' a todo el dataset
df["chordprog_funcional"] = df.apply(
    lambda row: transponer_progresion(row["chordprog"], row["key"]) if pd.notna(row["chordprog"]) else "",
    axis=1
)

El proceso es lento (12m 45.9s)

In [9]:
df.head()

,title,composedby,key,timesig,bars,chordprog,chordprog_funcional
0,Lullaby of Birdland,George Shearing,Ab,4 4,32,Fm7 Dm7b5 | G7b9 C7b9 | Fm7 DbM7 | Bbm7 Eb7 |\...,vi7 #ivø7 | VII7#5#32 III75#32 | vi7 ? | ? #V7...
1,It's A Most Unusual Day,Jimmy McHugh and HYarold Adamson,G,3 4,72,F#/G F#/G G | Em7 | Am7 | D7 |\nF#/G F#/G G | ...,VII VII I | vi7 | ii7 | V7 | VII VII I | vi7 |...
2,Jump Monk,Charles Mingus,Ab,4 4,54,Fm7 DbM7 | G7b5 C7 | Fm7 DbM7 | G7b5 C7 |\nFm7...,vi7 ? | VIIø75#3 III75#3 | vi7 ? | VIIø75#3 II...
3,Nuages,Django Reinhardt and Jacques Larme,G,4 4,32,Bbm7 Eb7 | Am7b5 D7b9 | G6 Am7 | Bm7 |\nBbm7 E...,? VI75#3 | iiø7b53 V753b2 | I65 ii7 | iii7 | ?...
4,Love Me Do,John Lennon and Paul McCartney,G,4 4,48,G | C | G | C |\nG | C | C | C |\nC | G | C | ...,I | IV | I | IV | I | IV | IV | IV | IV | I | ...


# Exploramos los resultados de la transcripción masiva con *music21*

In [15]:
from collections import Counter

# Concatenar todos los acordes funcionales en un único texto
# sin contar las battas de compas ' | '
todo_funcional = " ".join(
    df["chordprog_funcional"]
      .dropna()
      .str.replace(r"\s*\|\s*", " ", regex=True)  # reemplaza la barra (y espacios alrededor) por un espacio
      .str.strip()
)

# Contar símbolos
tokens = todo_funcional.split()
total = len(tokens)
errores = tokens.count("?")

porcentaje_errores = errores / total * 100
print(f"Total acordes: {total}")
print(f"Acordes no reconocidos (\"?\"): {errores}")
print(f"Porcentaje de no reconocidos: {porcentaje_errores:.2f}%")


Total acordes: 134339
Acordes no reconocidos ("?"): 23320
Porcentaje de no reconocidos: 17.36%


In [16]:
# Detectar si hay algún "?" en la progresión funcional
df["contiene_error"] = df["chordprog_funcional"].str.contains(r"\?", na=False)

# Resumen
total_canciones = len(df)
canciones_con_error = df["contiene_error"].sum()
canciones_sin_error = total_canciones - canciones_con_error
porcentaje_con_error = canciones_con_error / total_canciones * 100

print(f"🎵 Total canciones: {total_canciones}")
print(f"⚠️ Canciones con errores: {canciones_con_error} ({porcentaje_con_error:.2f}%)")
print(f"✅ Canciones sin errores: {canciones_sin_error}")


🎵 Total canciones: 2613
⚠️ Canciones con errores: 2048 (78.38%)
✅ Canciones sin errores: 565


In [17]:
# Función para contar cuántos "?" hay en cada progresión funcional
def contar_errores(prog):
    if isinstance(prog, str):
        return prog.split().count("?")
    return 0

# Aplicar al dataframe
df["num_errores"] = df["chordprog_funcional"].apply(contar_errores)

# Filtrar canciones con un máximo de 3 errores
df_tolerante = df[df["num_errores"] <= 3]

# Resultado
total_tolerante = len(df_tolerante)
print(f"🎵 Canciones con un máximo de 3 errores: {total_tolerante}")


🎵 Canciones con un máximo de 3 errores: 1045


# Guardamos el dataset con las columnas:
- title
- composedby
- key
- timesig
- bars
- chordprog: progresion original
- **chordprog_funcional**: transcripción funcional
- **contiene_error**: boolean
- **num_errores**: integer


In [ ]:
# df.to_csv("../../data/songdb_funcional_v1_mus21.csv", index=False)

## Posibles mejores:
- Tratamiento de tonalidades menores.
- Gestión de acordes no reconocidos.